<center><h1><b>CONVOLUTION NEURAL NETWORKS</b></h1></center>
We will now be evaluating the same dataset of mnist using CNNs.

In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using Theano backend.


In [3]:
batch_size = 128
num_classes = 10
epochs = 5

In [21]:
# input image dimensions
img_rows, img_cols = 28, 28

In [22]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Next we need to load the MNIST dataset and reshape it so that it is suitable for use training a CNN. In Keras, the layers used for two-dimensional convolutions expect pixel values with the dimensions [pixels][width][height].

In the case of RGB, the first dimension pixels would be 3 for the red, green and blue components and it would be like having 3 image inputs for every color image. In the case of MNIST where the pixel values are gray scale, the pixel dimension is set to 1.

In [23]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [24]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Convolution is the first layer to extract features from an input image. Here we define the window size, stride, and padding also. We can also use an activation function to introduce non linearity. 
<img src=cnn_fetaure.png>

Next we have defined a max pool layer. Pooling layers section would reduce the number of parameters when the images are too large.
<img src=maxpool.png>

Then we have added a dropout layer. Dropout consists in randomly setting a fraction rate of input units to 0 at each update during training time, which helps prevent overfitting.
 
We flatten the matrices into a column matrix and then feed into a softmax function, to get a final matrix predicting the classification of the image.


In [26]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
__________

In [28]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [29]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 275s 5ms/step - loss: 0.2753 - acc: 0.9150 - val_loss: 0.0537 - val_acc: 0.9824
Epoch 2/5
60000/60000 [==============================] - 275s 5ms/step - loss: 0.0889 - acc: 0.9741 - val_loss: 0.0491 - val_acc: 0.9845
Epoch 3/5
60000/60000 [==============================] - 275s 5ms/step - loss: 0.0659 - acc: 0.9802 - val_loss: 0.0350 - val_acc: 0.9884
Epoch 4/5
60000/60000 [==============================] - 281s 5ms/step - loss: 0.0553 - acc: 0.9836 - val_loss: 0.0312 - val_acc: 0.9886
Epoch 5/5
60000/60000 [==============================] - 272s 5ms/step - loss: 0.0475 - acc: 0.9857 - val_loss: 0.0296 - val_acc: 0.9903


In [30]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.02960577512978343
Test accuracy: 0.9903
